# Install and Import all the necessary packages and libraries

In [1]:
!pip install scikit-optimize
!pip install visualkeras
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10, cifar100
import numpy as np
import time
import math
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer
from skopt import Optimizer

from tensorflow.keras.utils import plot_model
import visualkeras
import cv2
import urllib.request
from google.colab.patches import cv2_imshow



# Definition of programmer's parameters (Controller's block)

In [3]:
#@markdown ##Dataset
topic_prefix = "cifar10" # @param ["cifar10", "cifar100", "mnist", "fashion_mnist"] {type:"string"}
if topic_prefix == "cifar10":
  dataset = cifar10.load_data()
elif topic_prefix == "cifar100":
  dataset = cifar100.load_data()
elif topic_prefix == "mnist":
  dataset = mnist.load_data()
elif topic_prefix == "fashion_mnist":
  dataset = fashion_mnist.load_data()

#@markdown ##Synopses-based Training Optimization Configuration
# Percentage of the dataset that will be used for training
sample_size_low = 0.05 #@param {type:"slider", min:0, max:0.99, step:0.05}
sample_size_high = 1 #@param {type:"slider", min:0, max:1, step:0.05}

# Percentage of the dataset that will be used for testing
perc_test = 1 #@param {type:"number"}

# The ID of the method that will be employed during sampling
# 0: Simple reservoir sampling in our initial training dataset
# 1: Reservoir sampling in each class based on the number of samples (per class)
sampling_method_id = 1 #@param {type:"integer"}

#@markdown ##NN Architecture Configuration
##################################################
# CONVOLUTIONAL LAYERS INPUT
##################################################
num_of_conv_layers_low = 1 #@param {type:"slider", min:1, max:5, step:1}
num_of_conv_layers_high = 4 #@param {type:"slider", min:1, max:5, step:1}

##################################################
# POOLING LAYERS INPUT
##################################################
num_of_pool_layers_low = 0 #@param {type:"slider", min:0, max:5, step:1}
num_of_pool_layers_high = 4 #@param {type:"slider", min:0, max:5, step:1}

##################################################
# DENSE LAYERS INPUT
##################################################
num_of_dense_layers_low = 0 #@param {type:"slider", min:0, max:5, step:1}
num_of_dense_layers_high = 4 #@param {type:"slider", min:1, max:5, step:1}

# Number of epochs for the training process
# One Epoch is when an ENTIRE (training) dataset is passed forward and backward through the neural network only once.
# NOTE: An epoch is comprised of one or more batches.
num_of_epochs_low = 1 #@param {type:"slider", min:1, max:30, step:1}
num_of_epochs_high = 10 #@param {type:"slider", min:1, max:30, step:1}

# The value for the learning rate for the training process (it is inserted to the Adam optimizer)
# During the optimization, the algorithm needs to take a series of tiny steps to descend the error mountain in order to minimize the error.
# The direction of the step is determined by the gradient, while the step size is determined by the learning rate.
lr_low = 1e-3 #@param {type:"number"}
lr_high = 1e-3 #@param {type:"number"}

# The size of the batch for the training process
# Total number of training examples present in a single batch.
# or, number of samples processed before the model is updated.
size_of_batch_low = 64 #@param {type:"integer"}
size_of_batch_high = 64 #@param {type:"integer"}

#@markdown ##BO and Tradeoff score Configuration
# Function to minimize over the gaussian prior
# It is used in the Bayesian Opt algorithm
acquisition_f = "gp_hedge" #@param ["LCB", "EI", "PI", "gp_hedge"]

num_of_BO_calls = 50 #@param {type:"slider", min:12, max:100, step:1}

# Maximum training time above which we cancel the experiment (in seconds)
# It is used in the score (accuracy--speed tradeoff) formula
theta_parameter = 10 #@param {type:"number"}

# Weight for the accuracy of the model. Max value: 0.99
# There is also the weight of training speed of the model (1 - lamda_acc)
# It is used in the score (accuracy--speed tradeoff) formula
lamda_acc = 0.5 #@param {type:"number"}


#@markdown ##Other Configuration(s)
CONV_NEURONS_CONST = 32 #@param {type:"integer"}
CONV_NEURONS_BOUND = 256 #@param {type:"integer"}
DENSE_NEURONS_CONST = 128 #@param {type:"integer"}
DENSE_NEURONS_BOUND = 32 #@param {type:"integer"}


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


# Dimensions of Optimization Problem

In [4]:
dim_sample_size = Real(low=sample_size_low, high=sample_size_high, name='sample_size')
dim_epochs_number = Integer(low=num_of_epochs_low, high=num_of_epochs_high, name='epochs_number')
# Code 1 -> conv, Code 2 -> pool, Code 3 -> dense, Code 4 -> dropout. We assume that the first layer is conv so dim1 is actualy dim2..
dim_layer_1 = Integer(low=1, high=3, name='dim_1')
dim_layer_2 = Integer(low=1, high=3, name='dim_2')
dim_layer_3 = Integer(low=1, high=3, name='dim_3')
dim_layer_4 = Integer(low=1, high=3, name='dim_4')
# dim_conv_number = Integer(low=num_of_conv_layers_low, high=num_of_conv_layers_high, name='conv_number')
# dim_pool_number = Integer(low=num_of_pool_layers_low, high=num_of_pool_layers_high, name='pool_number')
# dim_dense_number = Integer(low=num_of_dense_layers_low, high=num_of_dense_layers_high, name='dense_number')
# dim_lr = Real(low=lr_low, high=lr_high, name='learning_rate')
# dim_batch_size = Integer(low=size_of_batch_low, high=size_of_batch_high, name='batch_size')
#dim_neurons_number = Categorical(categories=lst_neurons_string, name='neurons_number')

dimensions = [dim_sample_size,
              dim_epochs_number,
              dim_layer_1,
              dim_layer_2,
              dim_layer_3,
              dim_layer_4
             ]

default_parameters = [[0.2, 3, 2, 1, 2, 3]]

# Load our image dataset

In [5]:
(train_images_all, train_labels_all), (test_images_all, test_labels_all) = dataset

# Get unique labels in our training dataset
unique_class_labels = np.unique(train_labels_all)

# Normalize pixel values in the trainning and the test datasets to be between 0 and 1
# TODO: parameter of normalization minmax scalar
train_images_all, test_images_all = train_images_all / 255.0, test_images_all / 255.0

if len(train_images_all.shape) == 3:
  # Make sure images have shape (28, 28, 1)
  train_images_all = np.expand_dims(train_images_all, -1)
  test_images_all = np.expand_dims(test_images_all, -1)
  print("x_train shape:", train_images_all.shape)
  print(train_images_all.shape[0], "train samples")
  print(test_images_all.shape[0], "test samples")

# Synopsis block of code

Optional: Check that our sampling method works appropriately

In [6]:
# A function that prints the occurence of each class in a list
def print_times_per_label(lst, labels_all):
  # Get unique labels in our training dataset
  unique_labels = np.unique(labels_all)
  for i in range(0, len(unique_labels)):
    print("Class", unique_labels[i], "has", lst.count(i), "samples in our dataset...")

**Reservoir Sampling Function**:
*Randomized algorithms for randomly choosing k samples from a list of n items, where n is either a very large or unknown number. Typically n is large enough that the list doesn’t fit into main memory.*

Algorithm implemented:


1.   Create an array reservoir[0,...,k-1] and copy first k items of stream[ ] to it.
2.   Now one by one consider all items from (k+1)th item to nth item.


> Steps

*   Generate a random number, denoted as j, from 0 to i where i is the index of the current item in stream[].
*   If j is in range 0 to k-1, replace reservoir[j] with stream[i]

In [7]:
# Select k items from a stream of items-data
import random

# A function to randomly select k items from stream[0..n-1].
def reservoir_sampling(stream, n, k):
  i = 0     # index for elements in stream[]

  # reservoir[] is the output array.
  # Initialize it with first k elements from stream[]
  reservoir = [0] * k

  for i in range(k):
    reservoir[i] = stream[i]

  # Iterate from the (k+1)th element to Nth element
  while(i < n):
    # Pick a random index from 0 to i.
    j = random.randrange(i+1)

    # If the randomly picked
    # index is smaller than k,
    # then replace the element
    # present at the index
    # with new element from stream
    if(j < k):
      reservoir[j] = stream[i]
    i+=1

  return reservoir

In [8]:
# A function that finds the size of each reservoir for every class depending on its occurence in the initial dataset
# and returns the unique labels that exist in our dataset along with the corresponding percentage
def reservoir_size_per_class(init_labels):

  # Get unique labels and their counts (how many times they appear) in our training dataset
  unique_labels, counts = np.unique(init_labels, return_counts = True)

  # Transform to list
  unique_labels_lst = unique_labels.tolist()
  counts_lst = counts.tolist()

  perc_per_class = []
  for i in range(len(unique_labels_lst)):
    perc_per_class.append(counts_lst[i]/len(init_labels))

  # print(perc_per_class)

  return perc_per_class, unique_labels_lst

"Pre-Processing Part (or Filtering)": Call the sampling mehod. Get samples from the training and the testing datasets.

In [9]:
def sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test):
  print("Percentage of filtering in our training dataset was set:")
  print(sample_size)
  if sampling_method_id == 0:
    # Simple reservoir sampling over the whole training dataset
    # Total size of the stream (or training dataset)
    n_train = len(train_images_all)

    # Number of samples that will be drawn
    k_train = int(n_train * sample_size)

    # Use the indexes of dataset in order to decide which samples will be drawn
    idx_tmp_train_list = list(range(0, n_train))

    # Find the indexes in order to construct the dataset that will be used during the training process
    idx_train = reservoir_sampling(idx_tmp_train_list, n_train, k_train)
  else:
    # Reservoir sampling in each class based on the number of samples (per class) that exist in the initial dataset
    # Find the size of each reservoir for every class depending on its occurence in the initial training dataset
    class_perc, unique_ids = reservoir_size_per_class(train_labels_all)

    # Stores the indexes (from all classes) in order to construct the dataset that will be used during the training process
    idx_train = []

    # Run for every single class the reservoir sampling seperately
    for i in range(0, len(unique_ids)):
      # Find the locations of each sample belonging to our class of interest
      tmp = np.where(train_labels_all == unique_ids[i])
      idx_of_class = tmp[0].tolist()

      # Run the reservoir sampling for the class of interest
      sampled_idx_of_class = reservoir_sampling(idx_of_class, len(idx_of_class), int(len(train_images_all) * sample_size * class_perc[i]))

      # Store the (sampled) samples from this class
      for j in range(0, len(sampled_idx_of_class)):
        idx_train.append(sampled_idx_of_class[j])

  # Store the corresponding images and labels from training dataset based on the sampled indexes
  train_images_lst = []
  for i in idx_train:
    train_images_lst.append(train_images_all[i])

  train_labels_lst = []
  for i in idx_train:
    train_labels_lst.append(train_labels_all[i])

  # Check the occurence of each class in the final training dataset
  print_times_per_label(train_labels_lst, train_labels_all)

  # Total size of the stream (or testing dataset)
  n_test = len(test_images_all)

  # Number of samples that will be drawn
  k_test = int(n_test * perc_test)

  # Use the indexes of dataset in order to decide which samples will be drawn
  idx_tmp_test_list = list(range(0, n_test))

  # Find the indexes in order to construct the dataset that will be used during the testing process
  idx_test = reservoir_sampling(idx_tmp_test_list, n_test, k_test)

  # Store the corresponding images and labels from testing dataset based on the sampled indexes
  test_images_lst = []
  for i in idx_test:
    test_images_lst.append(test_images_all[i])

  test_labels_lst = []
  for i in idx_test:
    test_labels_lst.append(test_labels_all[i])

  # Tranfsorm the lists that we stored our samples into arrays
  train_images = np.asarray(train_images_lst)
  train_labels = np.asarray(train_labels_lst)
  test_images = np.asarray(test_images_lst)
  test_labels = np.asarray(test_labels_lst)

  # Verify that the desired filtering was performed in both datasets
  print("Training dataset before sampling:")
  print(train_images_all.shape)
  print(train_labels_all.shape)
  print("Training dataset after sampling:")
  print(train_images.shape)
  print(train_labels.shape)

  print("Testing dataset before sampling:")
  print(test_images_all.shape)
  print(test_labels_all.shape)
  print("Testing dataset after sampling:")
  print(test_images.shape)
  print(test_labels.shape)

  return train_images, train_labels, test_images, test_labels

# CNN's architecture builder

Function that creates and returns a specific model, i.e., CNN architecture

In [10]:
def create_model(layer_lst, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):

    # The type of input based on the corresponding dataset
    dataset_shape = train_images_all.shape[1:]
    print(dataset_shape)
    model = models.Sequential()

    # STRATEGY FOR NUM_OF_NEURONS
    # Regarding CONV layers we begin the input of user and as we add conv layers we double the number of neurons, e.g., 1st -> 32, 2nd -> 64, 3rd -> 128 until we reach the upper bound (imput of user)
    # Regarding DENSE layers we begin the input of user and as we add conv layers we divide by 2 the number of neurons, e.g., 1st -> 128, 2nd ->64, 3rd -> 32 until we reach the lower bound (imput of user)

    # INTUITION for this strategy
    # The intution behind this is that early layers deal with primitive concepts and thus having a large amount of neurons wouldn't really benifit after some point,
    # but as you go deeper, the heierarchy of abstractions get richer and richer and you'd want to be able to capture as much information as you can and create
    # new-higher-richer abstaractions better. This is why you increase the neurons as you go deeper.
    # On the other hand, when you reach the end of the network, you'd want to choose the best features out of all the features you have so far developed, so you start
    # to gradually decrease the number of neurons so hopefully you'll end up with the most important features that matters to your specific task.

    conv_tmp = CONV_NEURONS_CONST
    dense_tmp = DENSE_NEURONS_CONST

    model.add(layers.Conv2D(int(CONV_NEURONS_CONST), (3, 3), activation='relu', input_shape = dataset_shape))
    conv_tmp = conv_tmp * 2
    for i in range(0, len(layer_lst)):
      if layer_lst[i] == 1:
        if conv_tmp <= CONV_NEURONS_BOUND:
          model.add(layers.Conv2D(conv_tmp, (3, 3), activation='relu'))
          conv_tmp = conv_tmp * 2
        else:
          model.add(layers.Conv2D(CONV_NEURONS_BOUND, (3, 3), activation='relu'))
          conv_tmp = conv_tmp * 2
      elif layer_lst[i] == 2:
        model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
      elif layer_lst[i] == 4:  # Dropout
            model.add(layers.Dropout(rate=0.5))
      else:
        flag = False
        for j in range(0, i):
          if layer_lst[j] == 3:
            flag = True
        if flag == False:
        #if layer_lst[i-1] == 1 or layer_lst[i-1] == 2 or layer_lst[i-1] == 4: # or layer_lst[i] == 3:
          model.add(layers.Flatten())
        if dense_tmp >= DENSE_NEURONS_BOUND:
          model.add(layers.Dense(dense_tmp, activation='relu'))
          dense_tmp = dense_tmp // 2
        else:
          model.add(layers.Dense(DENSE_NEURONS_BOUND, activation='relu'))

    # Softmax is an activation function that is used mainly for classification tasks
    # It normalizes the input vector into a probability distribution  that is proportional to the exponential of the input numbers.
    if 3 not in layer_lst:
      model.add(layers.Flatten())
    model.add(layers.Dense(len(unique_class_labels), activation='softmax'))

    model.compile(optimizer=Adam(1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model

    # ############################################################################################
    # # Part I: Convolutional part of our network, i.e., extraction of (important) features
    # ############################################################################################
    # if conv_number > pool_number:
    #   for i in range(0, int(conv_number) - int(pool_number)):
    #     if i == 0:
    #       model.add(layers.Conv2D(int(CONV_NEURONS_CONST), (3, 3), activation='relu', input_shape = dataset_shape))
    #       conv_tmp = conv_tmp * 2
    #     else:
    #       if conv_tmp <= CONV_NEURONS_BOUND:
    #         model.add(layers.Conv2D(conv_tmp, (3, 3), activation='relu'))
    #         conv_tmp = conv_tmp * 2
    #       else:
    #         model.add(layers.Conv2D(CONV_NEURONS_BOUND, (3, 3), activation='relu'))

    #   for i in range(int(conv_number) - int(pool_number), int(conv_number)):
    #     if conv_tmp <= CONV_NEURONS_BOUND:
    #       model.add(layers.Conv2D(conv_tmp, (3, 3), activation='relu'))
    #       conv_tmp = conv_tmp * 2
    #     else:
    #       model.add(layers.Conv2D(CONV_NEURONS_BOUND, (3, 3), activation='relu'))
    #     model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))

    # elif conv_number == pool_number:
    #   for i in range(0, int(conv_number)):
    #     if i == 0:
    #       model.add(layers.Conv2D(int(CONV_NEURONS_CONST), (3, 3), activation='relu', input_shape = dataset_shape))
    #       conv_tmp = conv_tmp * 2
    #       model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    #     else:
    #       if conv_tmp <= CONV_NEURONS_BOUND:
    #         model.add(layers.Conv2D(conv_tmp, (3, 3), activation='relu'))
    #         conv_tmp = conv_tmp * 2
    #       else:
    #         model.add(layers.Conv2D(CONV_NEURONS_BOUND, (3, 3), activation='relu'))
    #       model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    # else:
    #   for i in range(0, int(conv_number)):
    #     if i == 0:
    #       model.add(layers.Conv2D(int(CONV_NEURONS_CONST), (3, 3), activation='relu', input_shape = dataset_shape))
    #       conv_tmp = conv_tmp * 2
    #       model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    #     else:
    #       if conv_tmp <= CONV_NEURONS_BOUND:
    #         model.add(layers.Conv2D(conv_tmp, (3, 3), activation='relu'))
    #         conv_tmp = conv_tmp * 2
    #       else:
    #         model.add(layers.Conv2D(CONV_NEURONS_BOUND, (3, 3), activation='relu'))
    #       model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))

    #   for i in range(int(conv_number), int(pool_number)):
    #     model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))



    # ############################################################################################
    # # Part II: Dense part of our network, i.e., classification of an image in our classes
    # ############################################################################################

    # # Converts multi-dimensional matrix to single dimensional matrix.
    # model.add(layers.Flatten())

    # # Dense Layer is simple layer of neurons in which each neuron receives input from all the neurons of previous layer
    # for i in range(0, int(dense_number)):
    #   if i == 0:
    #     model.add(layers.Dense(int(DENSE_NEURONS_CONST), activation='relu'))
    #     dense_tmp = dense_tmp // 2
    #   else:
    #     if dense_tmp >= DENSE_NEURONS_BOUND:
    #       model.add(layers.Dense(dense_tmp, activation='relu'))
    #       dense_tmp = dense_tmp // 2
    #     else:
    #       model.add(layers.Dense(DENSE_NEURONS_BOUND, activation='relu'))


    # # Softmax is an activation function that is used mainly for classification tasks
    # # It normalizes the input vector into a probability distribution  that is proportional to the exponential of the input numbers.
    # model.add(layers.Dense(len(unique_class_labels), activation='softmax'))

    # model.compile(optimizer=Adam(1e-3),
    #           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    #           metrics=['accuracy'])

    # return model

# Bayesian Optimizer (using GPR)

The function that we want to minimize. It contains the sampling and the learning model. It evaluates our model with respect to the accuracy, training time and the score, i.e, the metric that captures the tradeoff between accuracy and training speed.

In [11]:
@use_named_args(dimensions = dimensions)
def fitness(sample_size, epochs_number, dim_1, dim_2, dim_3, dim_4):

    layer_lst = []
    layer_lst.append(dim_1)
    layer_lst.append(dim_2)
    layer_lst.append(dim_3)
    layer_lst.append(dim_4)

    print()
    print(layer_lst)
    print()

    flag = False
    for i in range(0, len(layer_lst)):
      if flag == False and layer_lst[i] == 3:
        flag = True
      if flag == True and (layer_lst[i] == 1 or layer_lst[i] == 2):
        print('Invalid Architecture!!!')
        extra_results.append([-1, -1])
        return 100000


    # Call the sampling method
    train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test)

    # Function that creates the model
    model = create_model(layer_lst, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND)

    model.summary()
    np.int = int
    start = time.time()

    blackbox = model.fit(x=train_images,
                        y=train_labels,
                        epochs=epochs_number,
                        batch_size=64
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

    # # Return the validation accuracy for the last epoch.
    # accuracy = blackbox.history['val_accuracy'][-1]

    # Compute the metric that captures the accuracy--speed tradeoff
    # Original formula
    # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Softmin
    # tradeOff_metric = 1 - np.log(1 + np.exp(-0.1 * test_acc) + np.exp(0.1 * min(1, tr_time/theta_parameter)))

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # F1 like
    # alpha = 2
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    # tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))

    # Print the results.
    print()
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)
    print()

    # Store the accuracy and the training speed of the corresponding model in order to be printed in the final cell
    tmp = [test_acc, tr_time]
    extra_results.append(tmp)

    # Delete the Keras model with these hyper-parameters from memory.
    del model

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    tf.compat.v1.reset_default_graph()

    return -tradeOff_metric

In [12]:
K.clear_session()
tf.compat.v1.reset_default_graph()

In [13]:
global extra_results
extra_results = []

Bayesian optimization using Gaussian Processes

In [14]:
start_program = time.time()

In [15]:
# Initialize the optimizer
opt = Optimizer(
    dimensions=dimensions,
    base_estimator="GP",
    acq_func=acquisition_f
)

# Initial points if you have them
if default_parameters:
    for x in default_parameters:
        opt.tell(x, fitness(x), fit = True)

# Convergence parameters
max_iter = 100  # Optional cap
convergence_window = 20  # Number of iterations to consider
tolerance = 1e-4  # Minimum improvement

prev_best = float('inf')
no_improve_counter = 0

for i in range(max_iter):
    next_x = opt.ask()
    next_y = fitness(next_x)
    opt.tell(next_x, next_y, fit = True)

    curr_best = np.min(opt.yi)
    improvement = prev_best - curr_best

    if improvement < tolerance:
        no_improve_counter += 1
    else:
        no_improve_counter = 0
        prev_best = curr_best

    if no_improve_counter >= convergence_window:
        print(f"Converged after {i+1} iterations.")
        break

# Result
best_index = np.argmin(opt.yi)
best_params = opt.Xi[best_index]
best_score = opt.yi[best_index]



[2, 1, 2, 3]

Percentage of filtering in our training dataset was set:
0.2
Class 0 has 1000 samples in our dataset...
Class 1 has 1000 samples in our dataset...
Class 2 has 1000 samples in our dataset...
Class 3 has 1000 samples in our dataset...
Class 4 has 1000 samples in our dataset...
Class 5 has 1000 samples in our dataset...
Class 6 has 1000 samples in our dataset...
Class 7 has 1000 samples in our dataset...
Class 8 has 1000 samples in our dataset...
Class 9 has 1000 samples in our dataset...
Training dataset before sampling:
(50000, 32, 32, 3)
(50000, 1)
Training dataset after sampling:
(10000, 32, 32, 3)
(10000, 1)
Testing dataset before sampling:
(10000, 32, 32, 3)
(10000, 1)
Testing dataset after sampling:
(10000, 32, 32, 3)
(10000, 1)
(32, 32, 3)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 422,218 (1.61 MB)

...

 Trainable params: 2,186 (8.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


771/771 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.2270 - loss: 2.1379
Epoch 2/6
771/771 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3685 - loss: 1.7458
Epoch 3/6
771/771 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4124 - loss: 1.6390
Epoch 4/6
771/771 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4368 - loss: 1.5836
Epoch 5/6
771/771 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4521 - loss: 1.5410
Epoch 6/6
771/771 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4674 - loss: 1.5022
313/313 - 2s - 6ms/step - accuracy: 0.4712 - loss: 1.4862

Accuracy (on the testing dataset): 47.12%
Training time:  15.479501008987427
0.5586290539015144

Converged after 31 iterations.


In [16]:
stop_program = time.time()

In [17]:
print(best_index)
print()
print("Sampling - Epochs")
print(best_params)
print()
print("Score")
print(best_score)

11

Sampling - Epochs
[0.7090143406641445, np.int64(9), np.int64(2), np.int64(2), np.int64(1), np.int64(3)]

Score
-0.586278041480575


In [18]:
print("Accuracy - training time")
extra_results[best_index]

Accuracy - training time


[0.6972000002861023, 19.251712560653687]

In [19]:
# gp_result = gp_minimize(func=fitness,
#                             dimensions = dimensions,
#                             n_calls = num_of_BO_calls,
#                             acq_func = acquisition_f,
#                             noise = "gaussian",
#                             n_jobs = -1,
#                             x0 = default_parameters)

# Results of Bayes Opt Algorithm

In [20]:
print(best_index)
print(best_params)
print(best_score * -1)

11
[0.7090143406641445, np.int64(9), np.int64(2), np.int64(2), np.int64(1), np.int64(3)]
0.586278041480575


In [21]:
opt.get_result()

          fun: -0.586278041480575
            x: [0.7090143406641445, np.int64(9), np.int64(2), np.int64(2), np.int64(1), np.int64(3)]
    func_vals: [-5.654e-01 -5.834e-01 ... -5.840e-01 -5.586e-01]
      x_iters: [[0.2, 3, 2, 1, 2, 3], [0.7113993934424395, np.int64(9), np.int64(2), np.int64(2), np.int64(1), np.int64(3)], [0.6362701179166663, np.int64(10), np.int64(2), np.int64(2), np.int64(3), np.int64(2)], [0.6892765020831237, np.int64(10), np.int64(2), np.int64(2), np.int64(3), np.int64(1)], [0.9877772782455418, np.int64(6), np.int64(2), np.int64(2), np.int64(2), np.int64(2)], [0.25491474326113184, np.int64(2), np.int64(2), np.int64(3), np.int64(2), np.int64(3)], [0.3079026381586052, np.int64(5), np.int64(2), np.int64(3), np.int64(1), np.int64(2)], [0.8935612560566217, np.int64(5), np.int64(2), np.int64(3), np.int64(1), np.int64(1)], [0.3297136911056411, np.int64(1), np.int64(1), np.int64(2), np.int64(3), np.int64(2)], [0.16310797374483377, np.int64(2), np.int64(2), np.int64(1), np

Inshights of BayesOpt algorithm in each iteration (or call)


In [22]:
# Compute the time
program_time = stop_program - start_program
program_time

522.1943120956421